# Digital Image Processing Exercise 5_3

Dr. Rahmati

By: Gholamreza Dar

Summer 2022

## Imports

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(); sns.set_style('dark')

import cv2


## Functions

In [ ]:
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def disp(img, title='', s=12, vmin=None, vmax=None):
    plt.figure(figsize=(s,s))
    plt.axis('off')
    if vmin is not None and vmax is not None:
        plt.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    else:
        plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.show()

## Loading Images

In [ ]:
img_a = rgb(cv2.imread('inputs/P3/cart_1.png'))
img_b = gray(rgb(cv2.imread('inputs/P3/cart_2.png'))).astype(np.float32)

img_a_hsv = cv2.cvtColor(img_a, cv2.COLOR_RGB2HSV)

disp(img_a[:,:,:], 'img_a_rgb')
disp(img_a[:,:,0], 'img_a_r')
disp(img_a[:,:,1], 'img_a_g')
disp(img_a[:,:,2], 'img_a_b')

print()

disp(img_a_hsv[:,:,:], 'img_a_hsv')
disp(img_a_hsv[:,:,0], 'img_a_h')
disp(img_a_hsv[:,:,1], 'img_a_s')
disp(img_a_hsv[:,:,2], 'img_a_v')

## Remove background

In [ ]:
# We apply median filter to remove the noise in the conveyor belt
img_a_median = cv2.medianBlur(img_a, 23)
disp(img_a_median, 'img_a_median')


In [ ]:
# Next let's do a color thresholding to extract the conveyor belt
# We use the HSV color space

img_a_background = cv2.inRange(img_a_hsv, (0,0.0,0), (1,0.2,100))
# print(img_a_hsv_thresh.shape)

# np.unique(img_a_hsv_thresh)

disp(img_a_background, 'img_a_background')

In [ ]:
# Apply median filter to remove the noise in the background
img_a_background_median = cv2.medianBlur(img_a_background, 11)
disp(img_a_background_median, 'img_a_background_median')

In [ ]:
# Remove the conveyor belt top and bottom stripes
h, w = img_a_background_median.shape
STRIPE_MARGIN = 55
img_a_clean = img_a_background_median[STRIPE_MARGIN:h-STRIPE_MARGIN, :]
disp(img_a_clean, 'img_a_clean')

In [ ]:
# Apply Opening to close white holes in objects
KERNEL_SIZE = 27
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (KERNEL_SIZE, KERNEL_SIZE))
img_a_open = cv2.morphologyEx(img_a_clean, cv2.MORPH_OPEN, kernel)
disp(img_a_open, 'img_a_open')

# To remove the small dots left in the image
img_a_open_close = cv2.morphologyEx(img_a_open, cv2.MORPH_CLOSE, kernel)
disp(img_a_open_close, 'img_a_open_close')


In [ ]:
# Now we have to measure the amount of pixels in each object using morphological operations
# We use the thresholding to extract the objects
img_a_open_close_thresh = cv2.threshold(img_a_open_close, 0, 255, cv2.THRESH_BINARY)[1]
disp(img_a_open_close_thresh, 'img_a_open_close_thresh')

# We use the connected components to measure the amount of pixels in each object
# We use the thresholding to extract the objects
# img_a_open_close_thresh_cc = cv2.connectedComponents(img_a_open_close_thresh)[1]
# disp(img_a_open_close_thresh_cc, 'img_a_open_close_thresh_cc')


In [ ]:
# Close the handle
KERNEL_SIZE = 5
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (KERNEL_SIZE, KERNEL_SIZE))
img_result = cv2.morphologyEx(img_a_open_close_thresh, cv2.MORPH_OPEN, kernel, iterations=11)
disp(img_result, 'handle removed')

In [ ]:
prices = [
    83,
    36,
    108,
    30,
    139,
    144,
    40,
    50,
    6.5,
    8,
    5,
    6
]

mihan_products = [0,0,0,1,0,0,0,0,0,1,0,1]
drinks = [0,0,0,1,0,0,0,0,1,0,0,0]
edible_products = [0,1,1,1,0,1,1,0,1,1,1,1]

POTATO_CHIPS_INDEX = 1
TINY_INDEX = 11

In [ ]:
img = img_result.copy()
disp(img)

In [ ]:
ITERS = 1
KERNEL_SIZE = 2

img = img_a_open_close_thresh.copy()
for i in range(200):
    img = cv2.morphologyEx(img, cv2.MORPH_DILATE, np.ones((KERNEL_SIZE, KERNEL_SIZE), np.float32), iterations=ITERS)
    # Count connected components and report it
    disp(img, f"Dilate {i*ITERS} | Components={0}",s=3)

In [ ]:
ITERS = 1
KERNEL_SIZE = 2

img = img_result.copy()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (KERNEL_SIZE, KERNEL_SIZE))
last_count = 100
for i in range(1000):
    img = cv2.morphologyEx(img, cv2.MORPH_DILATE, kernel, iterations=ITERS)
    # Count connected components and report it
    # ret_val, labels, stats, centeroids = cv2.connectedComponentsWithStats(img)
    # print(centeroids)

    cnts,hierarchy = cv2.findContours(img, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)

    
    if len(cnts)-1 < last_count:
        # print(f"Dilate {i*ITERS} | Components={len(cnts)-1}")
        # disp(img, f"Dilate {i*ITERS} | Components={len(cnts)-1}",s=3)
        last_count = len(cnts)-1

        img_rgb = np.repeat(img[:,:,np.newaxis], 3, axis=2)
        for c in cnts:
            cv2.drawContours(img_rgb, [c], 0, color=(255,0,0), thickness=20)
        disp(img_rgb,f"Dilate {i*ITERS} | Components={len(cnts)-1}",s=8)
        print(f"iter {i}", "items:\t", len(cnts)-1)
    # break

In [ ]:
hierarchy